In [ ]:
# import requests
# datasets = requests.get('https://api.census.gov/data.json')
# jdata = datasets.json()
# df = pd.DataFrame(jdata)
# # df = pd.concat([df.drop('dataset', axis=1), df.dataset.apply(pd.Series)], axis=1)
# df = df.dataset.apply(pd.Series)
# df = df[~df.description.str.contains('phased out')]
# df.loc[df['c_isTimeseries'] == True, 'c_vintage'] = df[df['c_isTimeseries'] == True].c_vintage.fillna('Time Series')
# # vintages = df.c_vintage.value_counts().reset_index().values
# df['code'] = df.identifier.str.split('/').apply(lambda x: x[-1]).astype(str)
# df['last_year'] = df.c_vintage.astype(str)
# df.to_pickle('datasets.pkl')



In [45]:
import pandas as pd

import os, sys
from PyQt5.QtWidgets import (QApplication, QCheckBox, QVBoxLayout,QPushButton,
                             QWidget, QLineEdit, QHBoxLayout, QGroupBox,
                             QMessageBox, QGridLayout, QListWidget, QLabel,
                            QPlainTextEdit)
from PyQt5.QtCore import QSize

df = pd.read_pickle('datasets.pkl')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 500)
g = df.groupby('last_year')
# codes_by_year = g.code.agg(lambda x: list(x)).to_dict()
titles_by_year = g.title.agg(lambda x: list(x)).to_dict()
# descriptions_by_year = g.description.agg(lambda x: list(x)).to_dict()
code_by_title = df[['title', 'code']].set_index('title').to_dict()
descriptions_by_title = df[['title', 'description']].set_index('title').to_dict()['description']

vintages = list(df.c_vintage.astype(str).unique())
vintages.sort()
vintages.reverse()

# if 'app' in globals():
#     del(app)
    
# app = QApplication([])    
# window = QWidget()


class interface:
    def __init__(self):
        self.app = QApplication([])    
        self.window = QWidget()

        self.mainLayout = QGridLayout()
        self.mainLayout.setRowStretch(0, 0)
        self.mainLayout.setRowStretch(1, 2)
        self.mainLayout.setRowStretch(2, 2)
        self.mainLayout.setColumnStretch(0, 0)
        self.mainLayout.setColumnStretch(1, 4)
        
        self.mainLayout.setColumnMinimumWidth(0, 100)
        self.window.setLayout(self.mainLayout)
        self.window.setGeometry(100, 100, 800, 600)

        self.create_key_slot( 0, 0)
#         self.create_series_slot( 0, 2)
#         self.create_series_slot( 1, 2)
        
        
        self.year_list_widget = QListWidget()
        self.year_list_widget.itemSelectionChanged.connect(self.datasets_list_clicked)
        self.create_year_list( 1, 0, 2, 1)
        
        self.data_series_list_widget = QListWidget()
        self.data_series_list_widget.itemSelectionChanged.connect(self.series_list_clicked)        
        self.create_dataset_list(0, 1, 2, 1)
        
        self.dataset_description = QPlainTextEdit('')
        self.create_dataset_description_box(2, 1, 1, 1)
        
        self.variable_list_widget = QListWidget()
        
#         self.variables_list_widget = QListWidget()
#         self.create_dataset_list( 1, 1)
        
        
#         self.mainLayout.cellRect (0, 0).setSize(QSize(300, 400))
#         self.mainLayout.cellRect (0, 1).setSize(QSize(300, 400))
#         self.mainLayout.cellRect (1, 0).setSize(QSize(1000, 1000))
#         self.mainLayout.cellRect (1, 1).setSize(QSize(1000, 1000))
        
        self.window.show()
        self.app.exec_()
        
    def create_key_slot(self, row, column):
        """Create the key spot and run the function to check if there is an API key
        on file"""        
    #     path = os.path.dirname(sys.argv[0]) +"\census_api_key.txt"
        path = os.getcwd()+"\\census_api_key.txt"
    #     print(path)

        def on_button_clicked():
            if len(api_slot.text())==40:
                with open(path, "w") as file:
                    file.write(api_slot.text())
            else:
                alert = QMessageBox()
                alert.setText('Key should be 40 characters long.')
                alert.exec_()


        new_group = QGroupBox()
        new_layout = QVBoxLayout()

        # Add api key entry box and check if there is a key already saved
        api_slot = QLineEdit(placeholderText = 'Put API key here')
        new_layout.addWidget(api_slot)
        if os.path.isfile(path):
            with open(path)as file:
                api_key = file.readlines()[0]
                api_slot.setText(api_key)
    #             print(api_key)


        save_button = QPushButton('Save Key')
        save_button.clicked.connect(on_button_clicked)
        new_layout.addWidget(save_button)

        new_group.setLayout(new_layout)
        self.mainLayout.addWidget(new_group, row, column, 1, 1)
    #     return new_group # for debugging purposes

    def create_year_list(self, row, column, rspan, cspan):
        new_group = QGroupBox()
        new_layout = QHBoxLayout()
        for year in vintages:
            self.year_list_widget.addItem(year)
        new_layout.addWidget(self.year_list_widget)
        new_group.setLayout(new_layout)
        self.mainLayout.addWidget(new_group, row, column, rspan, cspan)
    #     return new_group # for debugging purposes
    
    def create_variable_list(self, row, column, rspan, cspan):
        new_group = QGroupBox()
        new_layout = QHBoxLayout()
        new_layout.addWidget(self.variables_list_widget)
        new_group.setLayout(new_layout)
        self.mainLayout.addWidget(new_group, row, column, rspan, cspan)
        
    def create_dataset_list(self, row, column, rspan, cspan):
        new_group = QGroupBox()
        new_layout = QHBoxLayout()
        new_layout.addWidget(self.data_series_list_widget)
        new_group.setLayout(new_layout)
        self.mainLayout.addWidget(new_group, row, column, rspan, cspan)
    
    def create_dataset_description_box(self, row, column, rspan, cspan):
        new_group = QGroupBox()
        new_layout = QHBoxLayout()
        new_layout.addWidget(self.dataset_description)
        new_group.setLayout(new_layout)
        self.mainLayout.addWidget(new_group, row, column, rspan, cspan)
        
        
    def create_series_slot(self, row, column, rspan, cspan):
        new_group = QGroupBox()
        new_layout = QHBoxLayout()
    #    new_layout.addWidget(QLabel('Series Name'))
        new_layout.addWidget(QLineEdit(placeholderText = 'Series Name'))
        new_layout.addWidget(QCheckBox(""))
        new_group.setLayout(new_layout)
        self.mainLayout.addWidget(new_group, row, column, rspan, cspan)
    #     return new_group # for debugging purposes
    
    def datasets_list_clicked(self):
        selected_items = self.year_list_widget.selectedItems()
        if len(selected_items)>0:
            selected_text = selected_items[0].text()
            if selected_text in titles_by_year:                    
    #             print(selected)
                self.data_series_list_widget.clear()
                self.data_series_list_widget.addItems(titles_by_year[selected_text])
                self.data_series_list_widget.update()
            
    def series_list_clicked(self):
        selected_items = self.data_series_list_widget.selectedItems()
        if len(selected_items)>0:
            selected_text = selected_items[0].text()
            if selected_text in descriptions_by_title:
                self.dataset_description.clear()
                self.dataset_description.insertPlainText(descriptions_by_title[selected_text])
                self.current_code=code_by_title[selected_text]
                
            else:
                pass
        else:
            pass


if 'new_window' in globals():
    del(new_window)
new_window = interface()




In [23]:
import pandas as pd
df2 = pd.read_pickle('variable_reference.pkl')
df2.head()

,attributes,code,concept,datetime,group,variable,label,limit,predicateOnly,predicateType,required,title,values,variable_base
0,NaN,NONEMP2007,Selectable Geographies,NaN,N/A,COUNTY,FIPS County code,0,NaN,NaN,NaN,2007 Nonemployer Statistics: Non Employer Statistics,NaN,COUNTY
1,NaN,NONEMP2007,Selectable Geographies,NaN,N/A,GEOTYPE,Geography Type,0,NaN,string,NaN,2007 Nonemployer Statistics: Non Employer Statistics,NaN,GEOTY
2,NaN,NONEMP2007,Geographic Characteristics,NaN,N/A,GEO_ID,Geography Code,0,NaN,string,NaN,2007 Nonemployer Statistics: Non Employer Statistics,NaN,GEO_ID
3,NaN,NONEMP2007,Industry classification,NaN,N/A,NAICS2002,NAICS Industry Code,0,NaN,string,default displayed,2007 Nonemployer Statistics: Non Employer Statistics,NaN,NAICS2002
4,NaN,NONEMP2007,Industry classification,NaN,N/A,NAICS_TTL,Title of NAICS code,0,NaN,NaN,NaN,2007 Nonemployer Statistics: Non Employer Statistics,NaN,NAICS_TTL


In [27]:
variables_by_code = df2.groupby('code').variable_base.agg(lambda x: list(x)).to_dict()

# titles_by_year = g.title.agg(lambda x: list(x)).to_dict()

,0
code,
2000sf1,"[AIANHH, AIANHHFP, AIHHTLI, AITS, AITSCC, AITSC, ANPSADPI, ANRC, ANRCCC, AREALAND, AREAWATR, BLKGRP, BLOCK, CD106, CD108, CD109, CD110, CHARITER, CMSA, CONCIT, CONCITCC, CONCITSC, COUNTY, COUNTYSC, COUSUB, COUSUBCC, COUSUBSC, DIVISION, EXI, EXTCITIN, FILEID, FUNCSTAT, GCUNI, GEOCOMP, H001001, H002001, H002002, H002003, H002004, H002005, H002006, H003001, H003002, H003003, H004001, H004002, H004003, H005001, H005002, H005003, H005004, H005005, H005006, H005007, H006001, H006002, H006003, H006..."
2000sf3,"[AIANHH, AIANHHFP, AIHHTLI, AITS, AITSCC, AITSC, ANPSADPI, ANRC, ANRCCC, AREALAND, AREAWATR, CD106, CD108, CD109, CD110, CHARITER, CMSA, CONCIT, CONCITCC, CONCITSC, COUNTY, COUNTYSC, COUSUB, COUSUBCC, COUSUBC, COUSUBSC, DIVISION, D_H044_1, EXI, EXTCITIN, FILEID, FUNCSTAT, GCUNI, GEOCOMP, H001001, H002001, H002002, H002003, H003001, H004001, H004002, H005001, H005002, H005003, H005004, H005005, H005006, H005007, H006001, H006002, H006003, H007001, H007002, H007003, H008001, H008002, H008003, ..."
2012acs1,"[AIANHH, AIANHHFP, AIHHTLI, AITS, AITSC, ANRC, B00001, B00002, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B..."
2012acs3,"[AIANHH, AIANHHFP, AIHHTLI, AITS, AITSC, ANRC, B00001, B00002, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B01001A, B..."
2012acs3profile,"[AINDN, ANRC, BLKGRP, CBSA, CD, CSA, CTY, DIVISION, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02..."
ACSCD1132011,"[CDCURR, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, DP02, D..."
ACSCD1152015,"[CD, COMPONENT, DADSID, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, DP02PR, ..."
ACSCP1Y2010,"[AIANHH, ANRC, CBSA, CD111, CNECTA, COUNTY, COUSUB, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, CP02PR, 